# Run from bootstrap paths

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import math

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [2]:
old_store = paths.AnalysisStorage("weina_pre.nc")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [3]:
print "PathMovers:", len(old_store.pathmovers)
print "Engines:", len(old_store.engines)
print "Samples:", len(old_store.samples)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 12
Engines: 1
Samples: 50
Ensembles: 351
SampleSets: 1
Snapshots: 1784
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [4]:
template = old_store.snapshots[0]
engine = old_store.engines['default']
mstis = old_store.networks[0]
sset = old_store.tag['initial_sampleset']

In [5]:
sset.sanity_check()

## Running RETIS

Now we run the full calculation. Up to here, we haven't been storing any of our results. This time, we'll start a storage object, and we'll save the network we've created. Then we'll run a new `PathSampling` calculation object.

In [6]:
# logging creates ops_output.log file with details of what the calculation is doing
#import logging.config
#logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [7]:
storage = paths.storage.Storage("weina_production.nc", "w", template)

In [8]:
mstis_calc = paths.PathSampling(
    storage=storage,
    globalstate=sset,
    move_scheme=paths.DefaultScheme(mstis, engine)
)
mstis_calc.save_frequency = 10

Now everything is ready: let's run the simulation!

In [9]:
total_samples = 500
while len(storage.steps) < total_samples:
    try:
        mstis_calc.run_until(total_samples)
    except ValueError:
        pass

Working on Monte Carlo cycle number 28.



KeyboardInterrupt: 

In [10]:
print len(storage.steps)

28


In [11]:
# commented out during development, so we can "run all" and then do more
#storage.close()

In [12]:
#! skip
storage.close()